In [12]:
#!g1.1
import torch

In [13]:
#!g1.1
import pandas as pd
import numpy as np

In [14]:
#!g1.1

from torch.utils.data import Dataset, DataLoader, random_split

In [44]:
#!g1.1

from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.metrics import accuracy_score
from torchvision.datasets import MNIST
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torch.utils.data import DataLoader
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

EMNIST_NUM_CLASSES=26

class EMNISTCNN(Module):
    def __init__(self, n_channels):
        super(EMNISTCNN, self).__init__()
        # input to first hidden layer
        self.hidden1 = Conv2d(n_channels, 32, (3,3))
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # first pooling layer
        self.pool1 = MaxPool2d((2,2), stride=(2,2))
        # second hidden layer
        self.hidden2 = Conv2d(32, 32, (3,3))
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # second pooling layer
        self.pool2 = MaxPool2d((2,2), stride=(2,2))
        # fully connected layer
        self.hidden3 = Linear(5*5*32, 100)
        kaiming_uniform_(self.hidden3.weight, nonlinearity='relu')
        self.act3 = ReLU()
        # output layer
        self.hidden4 = Linear(100, EMNIST_NUM_CLASSES)
        xavier_uniform_(self.hidden4.weight)
        self.act4 = Softmax(dim=1)
 
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.pool1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.pool2(X)
        # flatten
        X = X.view(-1, 4*4*50)
        # third hidden layer
        X = self.hidden3(X)
        X = self.act3(X)
        X = self.hidden4(X)
        X = self.act4(X)
        return X

In [71]:
#!g1.1
class EMNISTCSVLoader(Dataset):
    def __init__(self, path, batch_size=1, shuffle=False, num_workers=0):
        super(EMNISTCSVLoader, self).__init__()
        df = pd.read_csv(path, header=None)
        self.X = df.values[:,1:].astype('float32') / 255
        self.X = self.X.reshape(-1, 1, 28, 28)
        self.Y = df.values[:,0]
        self.Y = self.Y.reshape(-1) - 1
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx: int):
        return [self.X[idx], self.Y[idx]]
    
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

In [72]:
#!g1.1
def prepare_letters_dataset(path):
    dataset = EMNISTCSVLoader(path, shuffle=True)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=32)
    return train_dl, test_dl

In [73]:
#!g1.1
model = EMNISTCNN(1)

In [74]:
#!g1.1
path = "emnist-letters-train.csv"
train_dl, test_dl = prepare_letters_dataset(path)

In [77]:
#!g1.1

def train_model(train_dl, model):
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(50):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()


def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        yhat = argmax(yhat, axis=1)
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

In [80]:
#!g1.1
def train_model(train_dl, model):
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(50):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()

In [81]:
#!g1.1
train_model(train_dl, model)

In [82]:
#!g1.1
evaluate_model(train_dl, model)

0.7245529111200752

In [ ]:
#!g1.1
